In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision

# torch.manual_seed(1)

In [2]:
EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = False

In [3]:
train_data = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)

In [4]:
test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000].cuda()/255.   # Tensor on GPU
test_y = test_data.test_labels[:2000].cuda()

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels=1, out_channels=16, 
                              kernel_size=5, stride=1, padding=2,),
                        nn.ReLU(), 
                        nn.MaxPool2d(kernel_size=2),)
        self.conv2 = nn.Sequential(
                        nn.Conv2d(16, 32, 5, 1, 2), 
                        nn.ReLU(), 
                        nn.MaxPool2d(2),)
        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [6]:
cnn = CNN()
cnn.cuda()      # Moves all model parameters and buffers to the GPU.

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)

In [7]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [8]:
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):

        # !!!!!!!! Change in here !!!!!!!!! #
        b_x = x.cuda()    # Tensor on GPU
        b_y = y.cuda()    # Tensor on GPU

        output = cnn(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            test_output = cnn(test_x)

            # !!!!!!!! Change in here !!!!!!!!! #
            pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()  # move the computation in GPU
            
            # change the dtype
            accuracy = torch.sum(pred_y == test_y).type(torch.FloatTensor)/2000
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.cpu(), '| test accuracy: %.2f' % accuracy)


Epoch:  0 | train loss: 2.2933 | test accuracy: 0.15
Epoch:  0 | train loss: 0.5384 | test accuracy: 0.79
Epoch:  0 | train loss: 0.2488 | test accuracy: 0.90
Epoch:  0 | train loss: 0.2262 | test accuracy: 0.90
Epoch:  0 | train loss: 0.2731 | test accuracy: 0.93
Epoch:  0 | train loss: 0.1069 | test accuracy: 0.94
Epoch:  0 | train loss: 0.0891 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1518 | test accuracy: 0.95
Epoch:  0 | train loss: 0.2976 | test accuracy: 0.96
Epoch:  0 | train loss: 0.1577 | test accuracy: 0.96
Epoch:  0 | train loss: 0.3434 | test accuracy: 0.97
Epoch:  0 | train loss: 0.1165 | test accuracy: 0.96
Epoch:  0 | train loss: 0.0549 | test accuracy: 0.97
Epoch:  0 | train loss: 0.1365 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0331 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0483 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0576 | test accuracy: 0.97
Epoch:  0 | train loss: 0.1473 | test accuracy: 0.97
Epoch:  0 | train loss: 0.1235 | test accuracy

In [9]:
test_output = cnn(test_x[:10])

# !!!!!!!! Change in here !!!!!!!!! #
pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze() # move the computation in GPU

print(pred_y, 'prediction number')
print(test_y[:10], 'real number')

tensor([ 7,  2,  1,  0,  4,  1,  4,  9,  5,  9], device='cuda:0') prediction number
tensor([ 7,  2,  1,  0,  4,  1,  4,  9,  5,  9], device='cuda:0') real number
